In [ ]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import os

#document loader
from langchain_community.document_loaders import PyPDFLoader

# vector store
from langchain_community.vectorstores import Chroma

#llm
from langchain_openai import OpenAI

In [3]:
loader = PyPDFLoader("/content/aws-overview.pdf") #upload the document you wish to use for chatbot
#https://d1.awsstatic.com/whitepapers/aws-overview.pdf
pages = loader.load()

In [4]:
len(pages)

85

In [5]:
pages[16]

Document(page_content='Overview of Amazon Web Services AWS Whitepaper\nSoftware Development Kits\nSoftware Development Kits\nOur Software Development Kits (SDKs) simplify using AWS services in your applications with an\nApplication Program Interface (API) tailored to your programming language or platform.\nAnalytics\nTopics\n•Amazon Athena (p. 10)\n•Amazon CloudSearch (p. 10)\n•Amazon Elasticsearch Service (p. 11)\n•Amazon EMR (p. 11)\n•Amazon FinSpace (p. 11)\n•Amazon Kinesis (p. 11)\n•Amazon Kinesis Data Firehose (p. 12)\n•Amazon Kinesis Data Analytics (p. 12)\n•Amazon Kinesis Data Streams (p. 12)\n•Amazon Kinesis Video Streams (p. 12)\n•Amazon Redshift (p. 12)\n•Amazon QuickSight (p. 13)\n•AWS Data Exchange (p. 13)\n•AWS Data Pipeline (p. 13)\n•AWS Glue (p. 13)\n•AWS Lake Formation (p. 14)\n•Amazon Managed Streaming for Apache Kafka (Amazon MSK) (p. 14)\nAmazon Athena\nAmazon Athena is an interactive query service that makes it easy to analyze data in Amazon S3 using\nstandard SQL. 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=500, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
  docs = text_splitter.split_documents(documents)
  return docs

In [7]:
new_pages = split_docs(pages)
len(new_pages)

738

In [8]:
new_pages[500].page_content

'history, and conﬁguration change notiﬁcations to enable security and governance. The Conﬁg Rules\nfeature enables you to create rules that automatically check the conﬁguration of AWS resources recorded\nby AWS Conﬁg.\nWith AWS Conﬁg, you can discover existing and deleted AWS resources, determine your overall\ncompliance against rules, and dive into conﬁguration details of a resource at any point in time. These'

In [9]:
new_pages[499].page_content

'the AWS Management Console, AWS SDKs, command line tools, and higher-level AWS services (such\nas AWS CloudFormation (p. 50)). The AWS API call history produced by CloudTrail enables security\nanalysis, resource change tracking, and compliance auditing.\nAWS Conﬁg\nAWS Conﬁg is a fully managed service that provides you with an AWS resource inventory, conﬁguration\nhistory, and conﬁguration change notiﬁcations to enable security and governance. The Conﬁg Rules'

In [10]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(new_pages, embedding_function)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain_together import Together


llm = Together(
    model="meta-llama/Llama-2-70b-chat-hf",
    max_tokens=256,
    temperature=0,
    top_k=1,
    together_api_key="ENTER YOUR API KEY"

    #https://api.together.ai/settings/api-keys
)

In [12]:
retriever = db.as_retriever(similarity_score_threshold = 0.9)

In [13]:
from langchain.prompts import PromptTemplate
prompt_template = """Please answer questions related AWS (Amazon web services). Try explaining in simple words. Answer in less than 100 words. If you don't know the answer simply respond as "Don't know man!"
 CONTEXT: {context}
 QUESTION: {question}"""

PROMPT = PromptTemplate(template = f"[INST] {prompt_template} [/INST]", input_variables=["context", "question"])

In [14]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='stuff',
    retriever= retriever,
    input_key = 'query',
    return_source_documents = True,
    chain_type_kwargs={"prompt":PROMPT},
    verbose=True

)

In [16]:
query = input()
response = chain(query)
response['result']

what is aws


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


"  AWS stands for Amazon Web Services. It's a cloud computing platform that provides a range of services for building, deploying, and managing applications and workloads in the cloud. With AWS, you can quickly spin up servers, databases, storage, and other resources to power your applications, and only pay for what you use. AWS offers a wide range of services, including computing, storage, databases, security, and machine learning, making it a popular choice for businesses of all sizes."

In [17]:
query = input()
response = chain(query)
response['result']

what is athena and redshift


> Entering new RetrievalQA chain...

> Finished chain.


'  Athena is a serverless, interactive query service that makes it easy to analyze data in Amazon Web Services (AWS). It allows users to run SQL queries against data stored in Amazon S3 and other data sources, and retrieve the results in a few seconds.\n\nRedshift is a fully managed, petabyte-scale data warehouse service that makes it fast, simple, and cost-effective to analyze all your data using standard SQL and your existing business intelligence (BI) tools. It is designed to handle complex queries and large datasets, and is often used for data warehousing and big data analytics.\n\nIn short, Athena is a service that allows users to run SQL queries against data stored in AWS, while Redshift is a data warehouse service that allows users to store, process, and analyze large amounts of data using SQL and BI tools.'

In [18]:
query = input()
response = chain(query)
response['result']


what is redshift


> Entering new RetrievalQA chain...

> Finished chain.


'  Redshift is a fully managed data warehouse service provided by Amazon Web Services (AWS). It allows users to store, process, and analyze large amounts of data using standard SQL and their existing business intelligence (BI) tools. Redshift is designed to be fast, simple, and cost-effective, making it a popular choice for organizations looking to migrate their data warehousing operations to the cloud.'

In [19]:
query = input()
response = chain(query)
response['result']



per hour rate of redshift?


> Entering new RetrievalQA chain...

> Finished chain.


"  Don't know man! The per hour rate of Redshift is not a standard or widely known metric, as it can vary depending on various factors such as the type and number of instances, storage, and data transfer. Additionally, Amazon Redshift pricing is based on a combination of factors, including the number of nodes, node types, and the amount of data stored. It's best to check the official AWS pricing website for the most up-to-date and accurate information."

In [20]:
query = input()
response = chain(query)
response['result']


amazon redshift per hour fees


> Entering new RetrievalQA chain...

> Finished chain.


'  Amazon Redshift charges $0.25 per hour with no commitments.'